In [ ]:
!pip install boto3 langchain langchain_community -U

In [3]:
import boto3

In [4]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
boto3_bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name='us-east-1'
)

In [6]:
llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                client = boto3_bedrock,
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                "stop_sequences":['</standalone_question>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

In [7]:
prompt = "Given the following conversation in <conversation></conversation>, and a follow up user question in <question></question>.\\n<conversation>\\nuser: aws clean room是什么？\\nAI: AWS Clean Rooms是一种安全的多方计算服务,允许不同组织在不共享原始数据的情况下,对彼此的数据进行联合分析。它采用差分隐私技术,确保数据隐私和安全。与其他类似服务相比,AWS Clean Rooms无需将数据移动到其他平台,可直接在AWS S3上进行分析。它还与AWS Data Exchange集成,方便寻找数据合作伙伴。AWS Clean Rooms将取代亚马逊广告的AMC服务,为更广泛的行业提供数据合作解决方案。\\n</conversation>\\n\\n<question>\\n他有什么优势？\\n</question>\\n\\nplease use the context in the chat conversation to rephrase the user question to be a standalone question, respond in the original language of user's question, don't translate the chat history and user question.\\nif you don't understand the user's question, or the question is not relevant to the conversation. please keep the orginal question.\\nSkip the preamble, don't explain, go straight into the answer. Please put the standalone question in <standalone_question> tag\\n"

In [8]:
llm_sonnet.invoke(prompt)

<standalone_question>AWS Clean Rooms相比其他类似服务有什么优势?

AIMessage(content='<standalone_question>AWS Clean Rooms相比其他类似服务有什么优势?')

In [10]:
template = """
You are a helpful assistant
Please answer users' question as possible as you can:
User question: {question}
"""


In [11]:
output_parser = StrOutputParser()
prompt_template = ChatPromptTemplate.from_template(template)

In [13]:
chain = prompt_template | llm_sonnet |output_parser

In [14]:
question = "who is the director of Terminator 2?"

In [15]:
chain.invoke({"question":question})

The director of Terminator 2: Judgment Day is James Cameron.

Some additional details:

- Terminator 2 was released in 1991 and was a sequel to the 1984 film The Terminator, which was also directed by James Cameron.

- It starred Arnold Schwarzenegger as the Terminator character, along with Linda Hamilton, Edward Furlong, and Robert Patrick.

- The film was a massive critical and commercial success, earning over $500 million at the box office against a $100 million budget.

- It was groundbreaking for its special effects and action sequences, especially the liquid metal T-1000 villain played by Robert Patrick.

- James Cameron co-wrote the screenplay along with William Wisher Jr. He won the Saturn Award for Best Director for the film.

- Terminator 2 is considered one of the greatest and most influential sequels ever made, as well as one of the best science fiction/action films of all time.

So in summary, the acclaimed director who helmed the blockbuster Terminator 2: Judgment Day was

'The director of Terminator 2: Judgment Day is James Cameron.\n\nSome additional details:\n\n- Terminator 2 was released in 1991 and was a sequel to the 1984 film The Terminator, which was also directed by James Cameron.\n\n- It starred Arnold Schwarzenegger as the Terminator character, along with Linda Hamilton, Edward Furlong, and Robert Patrick.\n\n- The film was a massive critical and commercial success, earning over $500 million at the box office against a $100 million budget.\n\n- It was groundbreaking for its special effects and action sequences, especially the liquid metal T-1000 villain played by Robert Patrick.\n\n- James Cameron co-wrote the screenplay along with William Wisher Jr. He won the Saturn Award for Best Director for the film.\n\n- Terminator 2 is considered one of the greatest and most influential sequels ever made, as well as one of the best science fiction/action films of all time.\n\nSo in summary, the acclaimed director who helmed the blockbuster Terminator 2: